In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [21]:
class AML(nn.Module):
    def __init__(self,D_in, D_out):
        super(AML, self).__init__()
        self.l1=nn.Linear(D_in,20)
        self.l2=nn.Linear(20,D_out)
        self.sig=nn.Sigmoid()
        
    def forward(self, x):
        x=self.l1(x)
        x=F.relu(x)
        x=self.l2(x)
        x=self.sig(x)
        return x

In [36]:
model=AML(9,2)
a=torch.ones(9)
model(a)

tensor([0.4817, 0.4149], grad_fn=<SigmoidBackward>)

In [37]:
df=pd.read_csv('data/full_ml_dataset.csv')
enc = LabelEncoder()
enc.fit(df.type.values)
df.type=enc.transform(df.type)
df.nameDest=df.nameDest.apply(lambda x: hash(x))
df.nameOrig=df.nameOrig.apply(lambda x: hash(x))
y=torch.Tensor(df.isFraud.values)
X=torch.Tensor(df.iloc[:,0:9].values)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
loader = TensorDataset(X, y)
data=DataLoader(loader, batch_size=128, shuffle=True)
for t in range(2):
    for X, y in data:
        y_pred = model(X)
        print(y.dtype, y_pred.dtype)
        loss = criterion(y_pred, y.long())
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()